In [1]:
%%capture

!pip install gdown

In [3]:
%%capture

import gdown

URL = 'https://drive.google.com/file/d/1v7phszU3p8oywmdh2t0GmS7I4BnJKYcD/view?usp=sharing'
gdown.download(URL, 'measurements.csv', quiet=True, fuzzy=True)

In [6]:
import pandas as pd

df = pd.read_csv('measurements.csv')
df.head(3)

,timestamp,constellation,svid,elevation,azimuth,cn0
0,2025-02-21 12:10:07,GPS,30,40,319,30.861214
1,2025-02-21 12:10:08,GPS,7,77,316,26.424671
2,2025-02-21 12:10:08,GPS,30,40,319,34.772774


In [12]:
df[['constellation', 'svid']].drop_duplicates().shape[0]

45

In [13]:
df['timestamp'].min()

'2025-02-21 12:10:07'

In [14]:
df['timestamp'].max()

'2025-02-21 12:12:48'

In [18]:
df[['constellation']].drop_duplicates()

,constellation
0,GPS
19,BeiDou
27,GLONASS
113,Galileo
114,QZSS


In [19]:
%%capture

import gdown

URL = 'https://drive.google.com/file/d/1BLX_HvpFZyBAjMfENqZzdsHRirJ-e0UG/view?usp=sharing'
gdown.download(URL, 'satellites.csv', quiet=True, fuzzy=True)

In [20]:
import pandas as pd

satellites_df = pd.read_csv('satellites.csv')
satellites_df.head(3)

,time,satellite_id,altitude,azimuth,satellite_type
0,2025-02-21T12:11:27.500Z,49336,19.596635,158.701902,QZSS
1,2025-02-21T12:11:27.500Z,42965,50.812158,90.386745,QZSS
2,2025-02-21T12:11:27.500Z,42917,55.675513,132.775134,QZSS


In [30]:
from scipy.spatial import cKDTree

tree = cKDTree(satellites_df[['altitude', 'azimuth']].values)
distances, indices = tree.query(df[['elevation', 'azimuth']].values)

df['satellite_id'] = satellites_df.iloc[indices]['satellite_id'].values
df['satellite_altitude'] = satellites_df.iloc[indices]['altitude'].values
df['satellite_azimuth'] = satellites_df.iloc[indices]['azimuth'].values

df['altitude_delta'] = df['elevation'] - df['satellite_altitude']
df['azimuth_delta'] = df['azimuth'] - df['satellite_azimuth']

df.drop_duplicates(subset=['constellation', 'svid'])[['constellation', 'svid', 'satellite_id']].to_csv('svid_to_norad_id.csv', index=False)